In this example we will build a very simple model that recognizes the color red and highlights it from the input image.

In [ ]:
import shutil
shutil.rmtree("examplemodel")

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
import sys, re
import glob
import matplotlib.image as mpimg

input_images = []
output_images = []

print("Loading images")
img_dir="color_image_pairs"
filelist = glob.glob(os.path.join(img_dir, '*.jpg'))
for imgFilename in sorted(filelist):
    if not imgFilename.endswith(".jpg"):
        continue

    a = mpimg.imread(imgFilename)
    if a is None:
        print("Unable to read image"+imgFilename)
        continue
    H, W = a.shape[:2]
    D = a.shape[2]
    WH = int(W/2)
    print("W/H"+str(W)+" / "+str(H)+"WH:"+str(WH))
    left = np.asarray(a[0:H, 0:WH])
    righ = np.asarray(a[0:H, WH:W])
    input_images.append(left)
    output_images.append(righ)

input_images  = np.asarray(input_images, dtype=np.float32)
output_images = np.asarray(output_images, dtype=np.float32)

print("Train data shape: "+ str(np.shape(input_images))+ " type=", str(input_images.dtype))
print("Labels shape: "+ str(np.shape(output_images))+ " type=", str(output_images.dtype))

#Another helper function
def showImage(aImg, aW, aH):
    image = aImg.astype(np.uint8)
    im = Image.fromarray(np.roll(image, 1, axis=-1))
    image = image.reshape((W,H,D))
    img = Image.fromarray(image, 'RGB')
    img.show()

Complete the convolution layer declaration below:

In [ ]:
def model_function(features, labels, mode):
  input = features["x"]
  input_layer = tf.reshape(input, [-1, W, H, D])
  conv = tf.layers.conv2d( inputs=, filters=, kernel_size=[, ], strides = , padding="same")
  pred_output = tf.reshape(conv, [-1, W, H, D]);

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=pred_output)

  output = labels
  outputGround = tf.reshape(output, [-1, W, H, D])
  loss = tf.losses.absolute_difference( labels = outputGround, predictions=pred_output)
  tf.summary.scalar("loss", loss)
  tf.summary.image('input', input_layer, max_outputs=4)
  tf.summary.image('pred output', pred_output, max_outputs=4)
  tf.summary.image('real output', outputGround, max_outputs=4)

  if mode == tf.estimator.ModeKeys.TRAIN:
      optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
      train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
      return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=logits)}
  return tf.estimator.EstimatorSpec( mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
estimator = tf.estimator.Estimator(model_fn=model_function, model_dir="examplemodel")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input_images},
    y=output_images,
    batch_size=6,
    num_epochs=None,
    shuffle=True)

estimator.train( input_fn=train_input_fn, steps = 10000)
print(str(estimator.get_variable_names()))

In [ ]:
def loadImage( aPath ):
    imgFilename = aPath
    if not imgFilename.endswith(".jpg"):
        print("ONLY JPEG FILES ACCEPTED")
    print("PATH: "+imgFilename)
    a = mpimg.imread(imgFilename)#1=color, 0=grayscale
    if a is None:
        print("Unable to read image"+imgFilename)
        print("image is null")
    return np.asarray(a, dtype=np.float32)

In [ ]:
print("Predict")
img = loadImage( "test_red.jpg" )
H, W = img.shape[:2]
D = img.shape[2]
HW = int(W/2)
print("Loaded image of shape "+str(img.shape))
test_data = img.reshape(1,W,H,D)
test_data = np.asarray(test_data, dtype=np.float32)
eval_input_fn = tf.estimator.inputs.numpy_input_fn( x={"x":test_data}, num_epochs=1, shuffle=False )
eval_results = estimator.predict(input_fn=eval_input_fn)
fin = eval_results.__next__()
print("result size: "+str(img.shape))
outData = fin.astype('uint8', casting='unsafe')
img = Image.fromarray(outData, 'RGB')
img.show()

In [ ]:
print(str(estimator.get_variable_names()))
print("WEIGHTS 0"+str(estimator.get_variable_value("conv2d/kernel")))
print("BIAS    0"+str(estimator.get_variable_value("conv2d/bias")))